In [1]:
import pandas as pd
# import vaex
import numpy as np
import glob
import dask.dataframe as dd
import json
from sklearn.model_selection import train_test_split
import math
import csv
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score, classification_report, confusion_matrix
import time
import _warnings
import tensorflow as tf
from tqdm import tqdm
import swifter
import argparse
import os
from copy import copy, deepcopy

2024-01-05 19:53:48.335752: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-05 19:53:48.385927: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-05 19:53:49.082392: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/tiendat/miniconda3/envs/torchtf/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import t

In [2]:
# Step 1: Import the file
df = pd.read_csv('../data.csv')
df.reset_index(inplace=True)
df.set_index('CONS_NO', inplace = True)
df

,index,FLAG,2014/1/1,2014/1/10,2014/1/11,2014/1/12,2014/1/13,2014/1/14,2014/1/15,2014/1/16,...,2016/9/28,2016/9/29,2016/9/3,2016/9/30,2016/9/4,2016/9/5,2016/9/6,2016/9/7,2016/9/8,2016/9/9
CONS_NO,,,,,,,,,,,,,,,,,,,,,
0387DD8A07E07FDA6271170F86AD9151,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10.12,9.96,16.92,7.60,27.22,18.05,26.47,18.75,17.84,14.92
01D6177B5D4FFE0CABA9EF17DAFC2B84,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4B75AC4F2D8434CFF62DB64D0BB43103,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B32AC8CC6D5D805AC053557AB05F5343,3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.50,9.99,11.78,18.59,26.80,18.57,14.59,12.82,19.37,15.92
EDFC78B07BA2908B3395C4EB2304665E,4,1,2.90,3.42,3.81,4.58,3.56,4.25,3.86,3.53,...,17.77,10.37,15.32,13.51,12.23,14.68,16.35,18.14,18.41,17.31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
F1472871E1AFF49D4289564B6377D76C,42367,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.25,3.56,3.38,4.39,3.72,3.77,3.96,3.64,3.40,4.38
F3C8BBCD2DC26C1E0249DEEF6A4256B7,42368,0,2.70,4.39,3.95,0.00,0.00,0.00,0.00,0.00,...,4.81,4.87,4.48,3.67,3.31,4.58,3.33,3.19,4.57,4.00
A9A0FE83467A680FBFB0DBFC910DF227,42369,0,0.58,0.84,1.61,0.90,0.60,0.82,0.89,0.03,...,NaN,0.66,2.92,2.36,3.86,4.28,3.37,6.67,2.44,1.15


In [3]:
# Step 2: Detect and remove rows with more than 600 NaNs
threshold = 600  # Set the threshold for NaN values
rows_to_drop = df[df.isnull().sum(axis=1) > threshold].index  # Get index of rows with more than 600 NaNs

# Step 3: Remove these rows from the DataFrame
df_cleaned = df.drop(index=rows_to_drop)
df_cleaned

,index,FLAG,2014/1/1,2014/1/10,2014/1/11,2014/1/12,2014/1/13,2014/1/14,2014/1/15,2014/1/16,...,2016/9/28,2016/9/29,2016/9/3,2016/9/30,2016/9/4,2016/9/5,2016/9/6,2016/9/7,2016/9/8,2016/9/9
CONS_NO,,,,,,,,,,,,,,,,,,,,,
0387DD8A07E07FDA6271170F86AD9151,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10.12,9.96,16.92,7.60,27.22,18.05,26.47,18.75,17.84,14.92
B32AC8CC6D5D805AC053557AB05F5343,3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.50,9.99,11.78,18.59,26.80,18.57,14.59,12.82,19.37,15.92
EDFC78B07BA2908B3395C4EB2304665E,4,1,2.90,3.42,3.81,4.58,3.56,4.25,3.86,3.53,...,17.77,10.37,15.32,13.51,12.23,14.68,16.35,18.14,18.41,17.31
34C1954AA3703C4F8BD8EAEA7C4B7B83,6,1,0.11,0.53,0.45,0.51,1.32,0.71,0.12,0.52,...,4.33,2.46,1.23,3.19,1.17,1.36,0.56,1.02,1.96,1.83
768309B0EB11FD436CEE5ABFB84F4C0C,7,1,0.91,0.86,1.10,0.66,5.82,3.17,1.18,4.05,...,2.36,2.82,2.28,3.70,3.07,4.61,3.13,3.10,3.74,3.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5FCE503B6B616D5023B3C46173383107,42366,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.49,0.77,0.00,0.64,0.00,1.29,0.67,0.76,1.16,0.40
F1472871E1AFF49D4289564B6377D76C,42367,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.25,3.56,3.38,4.39,3.72,3.77,3.96,3.64,3.40,4.38
F3C8BBCD2DC26C1E0249DEEF6A4256B7,42368,0,2.70,4.39,3.95,0.00,0.00,0.00,0.00,0.00,...,4.81,4.87,4.48,3.67,3.31,4.58,3.33,3.19,4.57,4.00


In [4]:
# Method 1
# Apply the linear interpolation method
# Assuming df is your DataFrame
df_interpolated = df_cleaned.interpolate(method='linear', axis=1, limit_direction='both')
# Replace NaN values with 0 where interpolation couldn't be applied
df_filled = df_cleaned.fillna(0)

df_filled


,index,FLAG,2014/1/1,2014/1/10,2014/1/11,2014/1/12,2014/1/13,2014/1/14,2014/1/15,2014/1/16,...,2016/9/28,2016/9/29,2016/9/3,2016/9/30,2016/9/4,2016/9/5,2016/9/6,2016/9/7,2016/9/8,2016/9/9
CONS_NO,,,,,,,,,,,,,,,,,,,,,
0387DD8A07E07FDA6271170F86AD9151,0,1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,10.12,9.96,16.92,7.60,27.22,18.05,26.47,18.75,17.84,14.92
B32AC8CC6D5D805AC053557AB05F5343,3,1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,6.50,9.99,11.78,18.59,26.80,18.57,14.59,12.82,19.37,15.92
EDFC78B07BA2908B3395C4EB2304665E,4,1,2.90,3.42,3.81,4.58,3.56,4.25,3.86,3.53,...,17.77,10.37,15.32,13.51,12.23,14.68,16.35,18.14,18.41,17.31
34C1954AA3703C4F8BD8EAEA7C4B7B83,6,1,0.11,0.53,0.45,0.51,1.32,0.71,0.12,0.52,...,4.33,2.46,1.23,3.19,1.17,1.36,0.56,1.02,1.96,1.83
768309B0EB11FD436CEE5ABFB84F4C0C,7,1,0.91,0.86,1.10,0.66,5.82,3.17,1.18,4.05,...,2.36,2.82,2.28,3.70,3.07,4.61,3.13,3.10,3.74,3.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5FCE503B6B616D5023B3C46173383107,42366,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.49,0.77,0.00,0.64,0.00,1.29,0.67,0.76,1.16,0.40
F1472871E1AFF49D4289564B6377D76C,42367,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,4.25,3.56,3.38,4.39,3.72,3.77,3.96,3.64,3.40,4.38
F3C8BBCD2DC26C1E0249DEEF6A4256B7,42368,0,2.70,4.39,3.95,0.00,0.00,0.00,0.00,0.00,...,4.81,4.87,4.48,3.67,3.31,4.58,3.33,3.19,4.57,4.00


In [70]:
df_pr = df_filled.drop(['index','FLAG'], axis = 1)
df_pr

,2014/1/1,2014/1/10,2014/1/11,2014/1/12,2014/1/13,2014/1/14,2014/1/15,2014/1/16,2014/1/17,2014/1/18,...,2016/9/28,2016/9/29,2016/9/3,2016/9/30,2016/9/4,2016/9/5,2016/9/6,2016/9/7,2016/9/8,2016/9/9
CONS_NO,,,,,,,,,,,,,,,,,,,,,
0387DD8A07E07FDA6271170F86AD9151,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,10.12,9.96,16.92,7.60,27.22,18.05,26.47,18.75,17.84,14.92
B32AC8CC6D5D805AC053557AB05F5343,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,6.50,9.99,11.78,18.59,26.80,18.57,14.59,12.82,19.37,15.92
EDFC78B07BA2908B3395C4EB2304665E,2.90,3.42,3.81,4.58,3.56,4.25,3.86,3.53,3.41,0.85,...,17.77,10.37,15.32,13.51,12.23,14.68,16.35,18.14,18.41,17.31
34C1954AA3703C4F8BD8EAEA7C4B7B83,0.11,0.53,0.45,0.51,1.32,0.71,0.12,0.52,0.55,0.74,...,4.33,2.46,1.23,3.19,1.17,1.36,0.56,1.02,1.96,1.83
768309B0EB11FD436CEE5ABFB84F4C0C,0.91,0.86,1.10,0.66,5.82,3.17,1.18,4.05,3.66,3.21,...,2.36,2.82,2.28,3.70,3.07,4.61,3.13,3.10,3.74,3.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5FCE503B6B616D5023B3C46173383107,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.49,0.77,0.00,0.64,0.00,1.29,0.67,0.76,1.16,0.40
F1472871E1AFF49D4289564B6377D76C,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,4.25,3.56,3.38,4.39,3.72,3.77,3.96,3.64,3.40,4.38
F3C8BBCD2DC26C1E0249DEEF6A4256B7,2.70,4.39,3.95,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,4.81,4.87,4.48,3.67,3.31,4.58,3.33,3.19,4.57,4.00


In [71]:
df_pr['data'] = df_pr[df_pr.columns].astype(float).values.tolist()
df_apr = df_pr['data']
df_apr

CONS_NO
0387DD8A07E07FDA6271170F86AD9151    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
B32AC8CC6D5D805AC053557AB05F5343    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
EDFC78B07BA2908B3395C4EB2304665E    [2.9, 3.42, 3.81, 4.58, 3.56, 4.25, 3.86, 3.53...
34C1954AA3703C4F8BD8EAEA7C4B7B83    [0.11, 0.53, 0.45, 0.51, 1.32, 0.71, 0.12, 0.5...
768309B0EB11FD436CEE5ABFB84F4C0C    [0.91, 0.86, 1.1, 0.66, 5.82, 3.17, 1.18, 4.05...
                                                          ...                        
5FCE503B6B616D5023B3C46173383107    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
F1472871E1AFF49D4289564B6377D76C    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
F3C8BBCD2DC26C1E0249DEEF6A4256B7    [2.7, 4.39, 3.95, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...
A9A0FE83467A680FBFB0DBFC910DF227    [0.58, 0.84, 1.61, 0.9, 0.6, 0.82, 0.89, 0.03,...
D9A6ADA018FA46A55D5438370456AA45    [16.89, 13.84, 13.5, 14.6, 14.46, 12.34, 15.37...
Name: data, Length: 31683, dtype: object

In [72]:
window_size = 28 # 4weeks
strided_size = 28
output_shape = ((1035 - window_size) // strided_size + 1, window_size)
print(output_shape)
def stride(x):
    as_strided = np.lib.stride_tricks.as_strided
    x = pd.Series(x)
    return as_strided(x, output_shape, (8*strided_size, 8))
data = df_apr.apply(stride)
data

(36, 28)


CONS_NO
0387DD8A07E07FDA6271170F86AD9151    [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
B32AC8CC6D5D805AC053557AB05F5343    [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
EDFC78B07BA2908B3395C4EB2304665E    [[2.9, 3.42, 3.81, 4.58, 3.56, 4.25, 3.86, 3.5...
34C1954AA3703C4F8BD8EAEA7C4B7B83    [[0.11, 0.53, 0.45, 0.51, 1.32, 0.71, 0.12, 0....
768309B0EB11FD436CEE5ABFB84F4C0C    [[0.91, 0.86, 1.1, 0.66, 5.82, 3.17, 1.18, 4.0...
                                                          ...                        
5FCE503B6B616D5023B3C46173383107    [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
F1472871E1AFF49D4289564B6377D76C    [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
F3C8BBCD2DC26C1E0249DEEF6A4256B7    [[2.7, 4.39, 3.95, 0.0, 0.0, 0.0, 0.0, 0.0, 0....
A9A0FE83467A680FBFB0DBFC910DF227    [[0.58, 0.84, 1.61, 0.9, 0.6, 0.82, 0.89, 0.03...
D9A6ADA018FA46A55D5438370456AA45    [[16.89, 13.84, 13.5, 14.6, 14.46, 12.34, 15.3...
Name: data, Length: 31683, dtype: object

In [73]:
df_dpr = data.to_frame()
df_dpr['Flag'] = df_filled['FLAG']
df_dpr

,data,Flag
CONS_NO,,
0387DD8A07E07FDA6271170F86AD9151,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",1
B32AC8CC6D5D805AC053557AB05F5343,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",1
EDFC78B07BA2908B3395C4EB2304665E,"[[2.9, 3.42, 3.81, 4.58, 3.56, 4.25, 3.86, 3.5...",1
34C1954AA3703C4F8BD8EAEA7C4B7B83,"[[0.11, 0.53, 0.45, 0.51, 1.32, 0.71, 0.12, 0....",1
768309B0EB11FD436CEE5ABFB84F4C0C,"[[0.91, 0.86, 1.1, 0.66, 5.82, 3.17, 1.18, 4.0...",1
...,...,...
5FCE503B6B616D5023B3C46173383107,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0
F1472871E1AFF49D4289564B6377D76C,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0
F3C8BBCD2DC26C1E0249DEEF6A4256B7,"[[2.7, 4.39, 3.95, 0.0, 0.0, 0.0, 0.0, 0.0, 0....",0


In [78]:
def serialize_example(x, y): 
    """converts x, y to tf.train.Example and serialize"""
    #Need to pay attention to whether it needs to be converted to numpy() form
    data = x
    data = tf.train.FloatList(value = np.array(data).flatten())
    label = tf.train.Int64List(value = np.array([y]))
    features = tf.train.Features(
        feature = {
            "data": tf.train.Feature(float_list = data),
            "label" : tf.train.Feature(int64_list = label)
        }
    )
    example = tf.train.Example(features = features)
    return example.SerializeToString()

def write_tfrecord(data, filename):
    tfrecord_writer = tf.io.TFRecordWriter(filename)
    for _, row in tqdm(data.iterrows()):
        X = (row['data'])
        Y = row['Flag']
        tfrecord_writer.write(serialize_example(X, Y))
    tfrecord_writer.close() 
    
write_tfrecord(df_dpr, '../data/data.tfrecord')

31683it [00:10, 3145.14it/s]


In [101]:
class TFwriter: 
    def __init__(self, outdir, start_idx = 0):
        print('Writing to: ', outdir)
        self._outdir = outdir
        self._start_idx = start_idx
    
    def serialize_example(self, x, y): 
        """converts x, y to tf.train.Example and serialize"""
        #Need to pay attention to whether it needs to be converted to numpy() form
        data = x
        data = tf.train.FloatList(value = np.array(data).flatten())
        label = tf.train.Int64List(value = np.array([y]).flatten())
        features = tf.train.Features(
            feature = {
                "data": tf.train.Feature(float_list = data),
                "Flag" : tf.train.Feature(int64_list = label)
            }
        )
        example = tf.train.Example(features = features)
        # print("EXAMPLE: ", example)
        return example.SerializeToString()

    def write(self, data, label):
        filename = os.path.join(self._outdir, str(self._start_idx)+'.tfrec')
        with tf.io.TFRecordWriter(filename) as outfile:
            outfile.write(self.serialize_example(data, label))
        self._start_idx += 1
        
def read_tfrecord(example, window_size):
    # window_size = 20
    feature_description = {
        'data': tf.io.FixedLenFeature([window_size], tf.float32),
        'Flag': tf.io.FixedLenFeature([1], tf.int64)
    }
    sample = tf.io.parse_single_example(example, feature_description)
    return sample

def write_tfrecord(dataset, tfwriter):
    for batch_data in iter(dataset):
        for x, y in zip(batch_data['data'], batch_data['Flag']):
            tfwriter.write(x, y)

data_dir = '../data/data.tfrecord'
train_dir = '../data/train'
val_dir = '../data/val'
if not os.path.exists(train_dir):
    os.makedirs(train_dir)
if not os.path.exists(val_dir):
    os.makedirs(val_dir)
    
train_writer = TFwriter(train_dir)
val_writer = TFwriter(val_dir)

train_ratio = 0.8 # TRAIN 80 / TEST 20
batch_size = 32

total_train_size = 0
total_val_size = 0

dataset = tf.data.TFRecordDataset("../data/data.tfrecord")
for batch_data in iter(dataset):
    print(batch_data)

# dataset = dataset.map(lambda x: read_tfrecord(x, 28), num_parallel_calls=tf.data.experimental.AUTOTUNE)
# dataset = dataset.shuffle(50000)

# for batch_data in iter(dataset):
#     print(batch_data)

# train_size = int(31683 * train_ratio)
# val_size = (31683 - train_size)

# train_dataset = dataset.take(train_size)
# val_dataset = dataset.skip(train_size)

# train_dataset = train_dataset.batch(batch_size)
# val_dataset = val_dataset.batch(batch_size)
    
# write_tfrecord(train_dataset, train_writer)
# write_tfrecord(val_dataset, val_writer)

# total_train_size += train_size
# total_val_size += val_size

# print('Total training: ', total_train_size)
# print('Total validation: ', total_val_size)

Writing to:  ../data/train
Writing to:  ../data/val
<TFRecordDatasetV2 element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>


2024-01-05 20:49:32.634571: W tensorflow/core/framework/op_kernel.cc:1828] OP_REQUIRES failed at example_parsing_ops.cc:98 : INVALID_ARGUMENT: Key: data.  Can't parse serialized Example.
2024-01-05 20:49:32.634621: W tensorflow/core/framework/op_kernel.cc:1828] OP_REQUIRES failed at example_parsing_ops.cc:98 : INVALID_ARGUMENT: Key: data.  Can't parse serialized Example.
2024-01-05 20:49:32.634695: W tensorflow/core/framework/op_kernel.cc:1828] OP_REQUIRES failed at example_parsing_ops.cc:98 : INVALID_ARGUMENT: Key: data.  Can't parse serialized Example.
2024-01-05 20:49:32.634730: W tensorflow/core/framework/op_kernel.cc:1828] OP_REQUIRES failed at example_parsing_ops.cc:98 : INVALID_ARGUMENT: Key: data.  Can't parse serialized Example.
2024-01-05 20:49:32.634786: W tensorflow/core/framework/op_kernel.cc:1828] OP_REQUIRES failed at example_parsing_ops.cc:98 : INVALID_ARGUMENT: Key: data.  Can't parse serialized Example.
2024-01-05 20:49:32.634852: W tensorflow/core/framework/op_kernel

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Key: data.  Can't parse serialized Example.
	 [[{{node ParseSingleExample/ParseExample/ParseExampleV2}}]] [Op:IteratorGetNext] name: 